In [1]:
import pandas as pd
import numpy as np
import pickle
import logging

from model_metrics import format_results
import data_clean_for_model
import PipelineHelper

/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
logging.basicConfig(level = logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
### 1. Load Data
df = pd.read_parquet("data/all_processed_df.parquet.gzip")

In [4]:
k = 5
rseed = 229
df["outcome"] = np.where( df["state"]=="successful", 1, 0, )
df["un_id"] = np.arange(0, df.shape[0], 1 )
df["name_len"] = df["name"].str.len()
df["cv_group"] = np.random.choice( np.arange(0, k), size=df.shape[0] )
df["binned_usd_goal"] = pd.qcut( np.log(df["usd_goal"]+1), 20 )

with open("model_config.json", 'r') as j:
     model_params = json.loads(j.read())
model_params['naive_bayes']['ngram_range'] = tuple(model_params['naive_bayes']['ngram_range'])

In [5]:
## load project metadata
logger.info("Loading features")
try:
    f = open("data/features.pkl", "rb")
    ft_dict = pickle.load(f)
    f.close()
    X_train, y_train, X_test, y_test = ft_dict.values()
except:
    X_train, X_test, y_train, y_test = data_clean_for_model.data_clean_for_model(df, "outcome", model_params, cv=model_params["cv"])

INFO:__main__:Loading features


In [6]:
# load text
logger.info("Processing text data")
blurb_train, blurb_test, _, _    = data_clean_for_model.process_blurb(df, model_params)

INFO:__main__:Processing text data


In [7]:
## 2. Run text models

# get naive bayes predictions
logger.info("Loading Naive Bayes predictions")
try:
    nb_proba_train = np.load("data/res/multi_nb_preds_train.npy")
    nb_proba_test = np.load("data/res/multi_nb_preds_test.npy")
except:
    logger.info("Running Naive Bayes model")
    nb_params = model_params['naive_bayes']
    nb_train_pred, nb_proba_train, nb_test_pred, nb_proba_test = PipelineHelper.naive_bayes_predictions(
        blurb_train, y_train, blurb_test,
        tfidf=nb_params['tf-idf'], ngram_range=nb_params['ngram_range']
    )
    np.save("data/res/multi_nb_preds_train.npy", nb_proba_train)
    np.save("data/res/multi_nb_preds_test.npy", nb_proba_test)

# get LDA topic model
logger.info("Loading LDA topic predictions")
try:
    lda_train = pd.read_csv("data/res/lda_train.csv").drop(columns=['Unnamed: 0'])
    lda_test = pd.read_csv("data/res/lda_test.csv").drop(columns=['Unnamed: 0'])
except:
    logger.info("Running LDA topic model")
    lda_params = model_params['lda_params']
    tokenized_train = blurb_train.apply(data_clean_for_model.tokenize_text)
    tokenized_test = blurb_test.apply(data_clean_for_model.tokenize_text)
    lda_train, lda_test = PipelineHelper.train_lda_model(tokenized_train, tokenized_test, params['lda'])
    lda_train.to_csv("data/res/lda_train.csv")
    lda_test.to_csv("data/res/lda_test.csv")

# get Word2Vec model predictions
logger.info("Loading Word2Vec dimension predictions")
try:
    f = open("data/res/w2v_dict.pkl", "rb")
    w2v_dict = pickle.load(f)
    f.close()
    w2v_train, w2v_test = w2v_dict.values()
except:
    raise Warning("Word2Vec function not implemented. Running without it -- likely will crash.")

INFO:__main__:Loading Naive Bayes predictions
INFO:__main__:Loading LDA topic predictions
INFO:__main__:Loading Word2Vec dimension predictions


In [8]:
### a. Just on metadata
logger.info("Getting metadata results")
stat_df, pred_df, models = PipelineHelper.run_analyses(X_train, y_train, X_test, y_test, model_params)

INFO:__main__:Getting metadata results
INFO:PipelineHelper:Fitting linear models
/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
INFO:PipelineHelper:Fitting ensemble models


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


In [10]:
### b. Just on metadata, - binned_usd_goal_outcome_mean
logger.info("Getting metadata - binned_usd_goal_outcome_mean results")
stat_df_nobinusd, pred_df_nobinusd, models_nobinusd = PipelineHelper.run_analyses(
    X_train.drop(columns=['binned_usd_goal_outcome_mean']), y_train, 
    X_test.drop(columns=['binned_usd_goal_outcome_mean']), y_test, model_params)

INFO:__main__:Getting metadata - binned_usd_goal_outcome_mean results
INFO:PipelineHelper:Fitting linear models
/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
INFO:PipelineHelper:Fitting ensemble models


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


In [15]:
### c. Just on metadata + nb 
logger.info("Getting metadata + naive bayes results")
X_train_nb = X_train.copy()
X_test_nb = X_test.copy()
# NB 
X_train_nb['nb_proba'] = nb_proba_train[:, 1]
X_test_nb['nb_proba'] = nb_proba_test[:, 1]
stat_df_nb, pred_df_nb, models_nb = PipelineHelper.run_analyses(X_train_nb, y_train, X_test_nb, y_test, model_params)

INFO:__main__:Getting metadata + naive bayes results
INFO:PipelineHelper:Fitting linear models
/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
INFO:PipelineHelper:Fitting ensemble models


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


In [18]:
### d. Just on metadata + nb + lda
logger.info("Getting metadata + naive bayes + LDA results")
X_train_nb_lda = pd.concat((X_train_nb, lda_train), axis=1)
X_test_nb_lda = pd.concat((X_test_nb, lda_test), axis=1)
stat_df_nb_lda, pred_df_nb_lda, models_nb_lda = PipelineHelper.run_analyses(X_train_nb_lda, y_train, X_test_nb_lda, y_test, model_params)

INFO:__main__:Getting metadata + naive bayes + LDA results
INFO:PipelineHelper:Fitting linear models
/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
INFO:PipelineHelper:Fitting ensemble models


In [20]:
### e. Just on metadata + nb + w2v
logger.info("Getting metadata + naive bayes + w2v results")
X_train_nb_w2v = pd.concat((X_train_nb, pd.DataFrame(w2v_train)), axis=1)
X_test_nb_w2v = pd.concat((X_test_nb, pd.DataFrame(w2v_test)), axis=1)
stat_df_nb_w2v, pred_df_nb_w2v, models_nb_w2v = PipelineHelper.run_analyses(X_train_nb_w2v, y_train, X_test_nb_w2v, y_test, model_params)

INFO:__main__:Getting metadata + naive bayes + w2v results
INFO:PipelineHelper:Fitting linear models
/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.15867e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
INFO:PipelineHelper:Fitting ensemble models


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


In [22]:
### f. Just on metadata + nb + lda - cols to drop 
logger.info("Getting metadata + naive bayes + LDA results")
cols_to_drop = [
    'dummy_cat_id_290', 'dummy_cat_id_300', 'dummy_cat_id_317','dummy_cat_id_386', 'dummy_cat_id_352', #'dummy_cat_id_1',
    'dummy_cat_id_355', 'dummy_cat_id_354', 'dummy_cat_id_321', 'dummy_cat_id_12', 'dummy_cat_id_340', 'dummy_cat_id_268', 'binned_usd_goal_outcome_mean'
]
stat_df_nb_lda_drop, pred_df_nb_lda_drop, models_nb_lda_drop = PipelineHelper.run_analyses(
    X_train_nb_lda.drop(columns=cols_to_drop), y_train, 
    X_test_nb_lda.drop(columns=cols_to_drop), y_test, model_params
)

INFO:__main__:Getting metadata + naive bayes + LDA results
INFO:PipelineHelper:Fitting linear models
/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
INFO:PipelineHelper:Fitting ensemble models


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


In [ ]:
stat_df.insert(0, "data", "metadata"), 
stat_df_nobinusd.insert(0, "data", "metadata_nobin"), 
stat_df_nb.insert(0, "data", "metadata_nb"),
stat_df_nb_lda.insert(0, "data", "metadata_nb_lda"),
stat_df_nb_w2v.insert(0, "data", "metadata_nb_w2v"),
stat_df_nb_lda_drop.insert(0, "data", "metadata_nb_lda_drop")

In [28]:
fin = (pd.concat((stat_df, stat_df_nobinusd, stat_df_nb, stat_df_nb_lda, stat_df_nb_w2v, stat_df_nb_lda_drop))
       .sort_values('accuracy', ascending=False)
       .assign(
           accuracy_rank = lambda x:np.arange(1, x.shape[0]+1, 1), 
           random_state = model_params['rseed']
       )
      )
fin.head()

,data,model,accuracy,f1_score,precision_1,precision_0,recall_1,recall_0,roc_auc,brier,accuracy_rank,random_state
0,metadata_nb,LGBMClassifier,0.825431,0.848953,0.885703,0.750599,0.815130,0.841002,0.828066,0.174569,1,229
0,metadata_nb_lda_drop,LGBMClassifier,0.823443,0.847502,0.882493,0.749517,0.815180,0.835932,0.825556,0.176557,2,229
0,metadata_nb_w2v,LGBMClassifier,0.823292,0.849325,0.872308,0.758067,0.827521,0.816899,0.822210,0.176708,3,229
0,metadata_nb_lda,LGBMClassifier,0.822885,0.846587,0.884252,0.747070,0.812001,0.839337,0.825669,0.177115,4,229
1,metadata_nb_lda,RandomForestClassifier,0.814192,0.838309,0.880065,0.734549,0.800335,0.835137,0.817736,0.185808,5,229


In [39]:
pred_df.columns = "metadata_" + pred_df.columns
pred_df_nobinusd.columns = "metadata_nobin_" + pred_df_nobinusd.columns
pred_df_nb.columns = "metadata_nb_" + pred_df_nb.columns
pred_df_nb_lda.columns = "metadata_nb_lda_" + pred_df_nb_lda.columns
pred_df_nb_w2v.columns = "metadata_nb_w2v_" + pred_df_nb_w2v.columns
pred_df_nb_lda_drop.columns = "metadata_nb_lda_drop_" + pred_df_nb_lda_drop.columns
pred_fin = pd.concat((pred_df, pred_df_nobinusd, pred_df_nb, pred_df_nb_lda, pred_df_nb_w2v, pred_df_nb_lda_drop), axis=1)

,metadata_metadata_LinearRegression_pred,metadata_metadata_Lasso_pred,metadata_metadata_Ridge_pred,metadata_metadata_LogisticRegression_pred,metadata_metadata_LGBMClassifier_pred,metadata_metadata_RandomForestClassifier_pred,metadata_nobin_LinearRegression_pred,metadata_nobin_Lasso_pred,metadata_nobin_Ridge_pred,metadata_nobin_LogisticRegression_pred,...,metadata_nb_w2v_Ridge_pred,metadata_nb_w2v_LogisticRegression_pred,metadata_nb_w2v_LGBMClassifier_pred,metadata_nb_w2v_RandomForestClassifier_pred,metadata_nb_lda_drop_LinearRegression_pred,metadata_nb_lda_drop_Lasso_pred,metadata_nb_lda_drop_Ridge_pred,metadata_nb_lda_drop_LogisticRegression_pred,metadata_nb_lda_drop_LGBMClassifier_pred,metadata_nb_lda_drop_RandomForestClassifier_pred
0,0.655400,0.832675,0.651824,1,1,1,0.655646,0.832675,0.652055,1,...,0.761006,1,1,1,0.800469,0.832675,0.803503,1,1,1
1,0.144957,0.473552,0.144069,1,0,0,0.138541,0.473552,0.137552,1,...,0.136281,1,0,0,0.072070,0.473552,0.079891,1,0,0
2,0.488107,0.425906,0.488423,1,0,0,0.479006,0.425906,0.479338,1,...,0.577682,1,1,1,0.581687,0.425906,0.579625,1,1,0
3,1.036096,0.832221,1.037228,1,1,1,1.027969,0.832221,1.028984,1,...,1.201803,1,1,1,1.107628,0.832221,1.102822,1,1,1
4,0.154730,0.290907,0.154283,0,0,0,0.192137,0.290907,0.191644,0,...,0.140261,0,0,0,0.278180,0.290907,0.279277,0,0,0
